In [5]:
import networkx as nx
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Step 1: Load Data (Make Sure the File is in the Same Directory)
network_data = pd.read_csv("networks_assignment.csv")

# Step 2: Extract Nodes & Edges
source_nodes = network_data["LABELS"]  # Row Labels (D, F, I, N, S)
target_nodes = list(network_data.columns[1:])  # All other columns are target nodes

edges = []
for _, row in network_data.iterrows():
    source = row["LABELS"]
    for target in target_nodes:
        weight = row[target]
        if weight > 0:  # Only create edges where there is a connection
            edges.append((source, target, weight))

# Step 3: Define Node Groups Based on Colors
pentagram_nodes = {"D", "F", "I", "N", "S"}
color_mapping = {
    "Blue": pentagram_nodes,
    "Green": {'BIH', 'GEO', 'ISR', 'MNE', 'SRB', 'CHE', 'TUR', 'UKR', 'GBR', 'AUS', 'HKG', 'USA'},
    "Yellow": {'AUT', 'BEL', 'BGR', 'HRV', 'CZE', 'EST', 'FRA', 'DEU', 'GRC', 'HUN', 'IRL', 'ITA', 'LVA', 'LUX', 'NLD', 'PRT', 'ROU', 'SVK', 'SVN', 'ESP'}
}

# Step 4: Assign Colors Based on Classification
nodes = set(source_nodes).union(set(target_nodes))
node_colors = {node: "gray" for node in nodes}  # Default gray
for color, group in color_mapping.items():
    for node in group:
        if node in nodes:
            node_colors[node] = color.lower()  # Convert to lowercase for Plotly

# Step 5: Create Graph & Add Edges
G = nx.Graph()
for source, target, weight in edges:
    G.add_edge(source, target, weight=weight)

# Step 6: Explicitly Position Pentagram Nodes
pentagram_positions = {
    "D": (0, 1), "F": (-0.951, 0.309), "I": (-0.587, -0.809),
    "N": (0.587, -0.809), "S": (0.951, 0.309)
}

# Assign dynamic positions for all other nodes
spring_positions = nx.spring_layout(G, seed=42)
for node in G.nodes():
    if node not in pentagram_positions:
        pentagram_positions[node] = spring_positions[node]  # Assign spring layout positions

# Step 7: Create Plotly Edges with Endpoints
edge_x, edge_y, endpoint_x, endpoint_y = [], [], [], []
for edge in G.edges():
    x0, y0 = pentagram_positions[edge[0]]
    x1, y1 = pentagram_positions[edge[1]]
    
    # Edge line
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])
    
    # Small black endpoint at the connection
    endpoint_x.append(x1)
    endpoint_y.append(y1)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=1, color='black'),
    hoverinfo='none',
    mode='lines'
)

# Step 8: Small Black Endpoints
endpoint_trace = go.Scatter(
    x=endpoint_x, y=endpoint_y,
    mode='markers',
    marker=dict(size=5, color='black'),
    hoverinfo='none'
)

# Step 9: Create Plotly Nodes
node_x, node_y, colors, labels = [], [], [], []
for node in G.nodes():
    x, y = pentagram_positions[node]
    node_x.append(x)
    node_y.append(y)
    colors.append(node_colors.get(node, "gray"))  # Use mapped colors, default gray
    labels.append(node)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    text=labels,
    textposition="middle center",
    marker=dict(size=40, color=colors, line=dict(color='white', width=1)),  # Larger circles
    textfont=dict(color='white')  # Change text color to white
)

# Step 10: Create Figure
fig = go.Figure(data=[edge_trace, endpoint_trace, node_trace])
fig.update_layout(
    showlegend=False,
    title="Network Graph",
    plot_bgcolor='white',  
    xaxis=dict(showgrid=False, zeroline=False, visible=False), 
    yaxis=dict(showgrid=False, zeroline=False, visible=False)
)

# Step 11: Show Graph
fig.show()
